In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import keras
from keras.layers import LSTM, Embedding, Dense, Flatten, Dropout, Add, Multiply
from keras.layers import Input, LayerNormalization
from keras import Sequential
from keras import Model
from keras.models import save_model, load_model

from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

from sklearn.model_selection import train_test_split

from tensorflow.keras.metrics import RootMeanSquaredError
import math

In [2]:
import data_preprocessing

In [3]:
def remove_nan(df_node):
    mean = np.nanmean(df_node['speed'])
    df_node['speed'] = df_node['speed'].replace()
    return df_node

In [4]:
def av_time(row):
    temp_df = nodes_df[nodes_df['Id'] ==row["Id"]]
    time = (temp_df['distance']/(temp_df['speed']))
    total_time = sum(time)/1000
    return total_time

In [5]:
def av_speed(row):
    return row["route_distance_km"]/row["av_time_H"]

In [6]:
def num_nods(row):
    temp_df = nodes_df[nodes_df['Id'] ==row["Id"]]
    return len(temp_df)

In [97]:
a = 0
for i in df["Id"]:
    b = len(nodes_df[nodes_df['Id'] == i])
    if b>a:
        a = b
a

272

In [93]:
df

,Id,running_time,completed_time,route_distance_km,delta_time,time,month,quarter,day_week,weekend,minute,hour,av_time_H,num_nods,av_speed_kmH
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,2022-01-24 18:30:21,1,1,0,0,30.0,18.0,0.123829,106,30.202871
1,-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,2022-01-24 06:53:53,1,1,0,0,53.0,6.0,0.094348,61,37.372254
2,-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,2022-01-24 10:00:59,1,1,0,0,0.0,10.0,0.154315,151,32.861256
3,5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,2022-01-24 14:28:05,1,1,0,0,28.0,14.0,0.090993,69,31.507933
4,1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,2022-01-24 11:57:29,1,1,0,0,57.0,11.0,0.114511,81,32.756669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,7096714159023973792,2022-01-24 21:10:38,2022-01-24 21:23:53,7.397,795.0,2022-01-24 21:10:38,1,1,0,0,10.0,21.0,0.152364,114,48.548230
4996,-3836026425968071806,2022-01-24 15:10:27,2022-01-24 15:20:21,1.948,594.0,2022-01-24 15:10:27,1,1,0,0,10.0,15.0,0.095044,40,20.495674
4997,2926216435675216636,2022-01-24 13:57:04,2022-01-24 14:03:18,2.547,374.0,2022-01-24 13:57:04,1,1,0,0,57.0,13.0,0.074323,60,34.269213
4998,-6677307480063489707,2022-01-24 08:46:13,2022-01-24 08:55:34,3.013,561.0,2022-01-24 08:46:13,1,1,0,0,46.0,8.0,0.085984,71,35.041395


In [18]:
orders_df = pd.read_csv('../../data/initial/orders.csv')

In [9]:
orders_df_test = pd.read_csv('../../data/initial/final_test.csv')

In [270]:
time_columns = ["month", "quarter", "day_week", "weekend", "minute", "hour"]

In [271]:
static_columns = ["route_distance_km", "av_time_H", "av_speed_kmH", "num_nods"]

In [14]:
df = data_preprocessing.date_transform(orders_df)

In [15]:
df_test =  data_preprocessing.date_transform(orders_df_test)

In [96]:
nodes_df = pd.read_csv('../../data/initial/nodes.csv')

In [23]:
nodes_df = remove_nan(nodes_df)

In [24]:
df["av_time_H"] = df.apply(av_time, axis=1)

In [25]:
df["num_nods"] = df.apply(num_nods, axis=1)

In [26]:
df["av_speed_kmH"] = df.apply(av_speed, axis=1)

In [297]:
nodes_df = pd.read_csv('../../data/initial/nodes_test.csv')

In [298]:
nodes_df = remove_nan(nodes_df)

In [299]:
df_test["av_time_H"] = df_test.apply(av_time, axis=1)

In [300]:
df_test["av_speed_kmH"] = df_test.apply(av_speed, axis=1)

In [301]:
df_test["num_nods"] = df_test.apply(num_nods, axis=1)

In [237]:
x = df.drop(['completed_time', 'running_time', "time", 'delta_time'], axis = 1)

In [238]:
len(x.columns)

11

In [239]:

#x_test = df_test.drop(['Id','completed_time', 'running_time', "time", 'delta_time'], axis = 1)

In [240]:
x.columns

Index(['Id', 'route_distance_km', 'month', 'quarter', 'day_week', 'weekend',
       'minute', 'hour', 'av_time_H', 'num_nods', 'av_speed_kmH'],
      dtype='object')

In [241]:
y = df['delta_time']

In [302]:
x_test = df_test.drop(['running_time', "time"], axis = 1)

In [243]:
#x_test = np.asarray(x_test).astype('float32')
#y_test = np.asarray(y_test).astype('float32')

In [244]:
X_train, Y_train = x, y

In [275]:


static_x_train = X_train.drop(time_columns, axis = 1)
time_x_train = X_train.drop(static_columns, axis = 1)

In [277]:
time_x_train

,month,quarter,day_week,weekend,minute,hour
0,1,1,0,0,30.0,18.0
1,1,1,0,0,53.0,6.0
2,1,1,0,0,0.0,10.0
3,1,1,0,0,28.0,14.0
4,1,1,0,0,57.0,11.0
...,...,...,...,...,...,...
4995,1,1,0,0,10.0,21.0
4996,1,1,0,0,10.0,15.0
4997,1,1,0,0,57.0,13.0
4998,1,1,0,0,46.0,8.0


In [274]:
#static_x_train = static_x_train.drop(["Id"], axis=1)
#time_x_train = time_x_train.drop(["Id"], axis=1)

In [246]:
lstm_x_speed = []
lstm_x_dist = []

In [247]:
for i in X_train["Id"]:
    temp_df = nodes_df[nodes_df['Id'] == i]
    ls = temp_df["speed"].to_numpy()
    ls = np.pad(ls, (0, 300 - len(ls)%300), 'constant')
    lstm_x_speed.append(ls)
    ld = temp_df["distance"].to_numpy()
    ld = np.pad(ld, (0, 300 - len(ld)%300), 'constant')
    lstm_x_speed.append(ls)
    lstm_x_dist.append(ld)

In [248]:
lstm_x_speed = np.asarray(lstm_x_speed).astype('float32')
lstm_x_dist = np.asarray(lstm_x_dist).astype('float32')


In [249]:
x= x.drop(["Id"], axis=1)

In [251]:
static_x_train = np.asarray(static_x_train).astype('float32')
static_x_test = np.asarray(static_x_test).astype('float32')

time_x_train = np.asarray(time_x_train).astype('float32')
time_x_test = np.asarray(time_x_test).astype('float32')

In [252]:
Y_test = np.asarray(Y_test).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')

In [279]:
optimizer = keras.optimizers.Adam(learning_rate=0.04)

In [280]:
input_layer_lstm_s = Input(shape=(300,))
embedded_s = Embedding(1, output_dim=32, input_length=300)(input_layer_lstm_s)

input_layer_lstm_d = Input(shape=(300,))
embedded_d = Embedding(1, output_dim=32, input_length=300)(input_layer_lstm_d)

In [281]:
lstm_s = LSTM(units=32)(embedded_s)
lstm_s =Dense(128)(lstm_s)

lstm_d = LSTM(units=32)(embedded_d)
lstm_d =Dense(128)(lstm_d)

In [282]:
input_layer_time = Input(shape=(len(time_columns),), name='input_t')
input_layer_static = Input(shape=(len(static_columns),), name='input_s', dtype='float32')

In [283]:
x_time = Dense(128)(input_layer_time)
x_static = Dense(128)(input_layer_static)

x_2_t = Dense(128)(input_layer_time)
x_2_s  = Dense(128)(input_layer_static)

In [284]:
combined = Add()([x_2_t, x_2_s])
combined = LayerNormalization()(combined)

In [285]:
combined_FCNN = Dense(128)(combined)
combined_FCNN_2 = Dense(128)(combined_FCNN)

In [286]:
connected_lstm = Multiply()([lstm_s, lstm_d])

In [287]:
combined_FCNN_2 = Add()([combined_FCNN_2, connected_lstm])

In [288]:

combined_FCNN_2N = LayerNormalization()(combined_FCNN_2)
combined_FCNN_3 = Dense(128)(combined_FCNN_2N)
combined_FCNN_3N = LayerNormalization()(combined_FCNN_3)

combined_FCNN_3N = Add()([combined_FCNN_3N, connected_lstm])

combined_FCNN_4 = Dense(256)(combined_FCNN_3N)

combined_FCNN_4N = LayerNormalization()(combined_FCNN_4)

combined_FCNN_4 = Dense(64)(combined_FCNN_4)
dropout = Dropout(0.2)(combined_FCNN_4)

In [289]:
output = Dense(1)(dropout)

In [290]:
model = Model(inputs=[input_layer_time, input_layer_static, input_layer_lstm_s, input_layer_lstm_d], outputs=output)

In [291]:
model.compile(loss="mean_absolute_error", metrics = ['mse'], optimizer=optimizer)

In [292]:

model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_t (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 input_s (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_104 (Dense)              (None, 128)          896         ['input_t[0][0]']                
                                                                                                  
 dense_105 (Dense)              (None, 128)          640         ['input_s[0][0]']                
                                                                                            

In [293]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [294]:
history = model.fit(
    [time_x_train, static_x_train, lstm_x_speed, lstm_x_dist],
    Y_train,
    epochs=45,
    verbose=1,
    validation_split = 0.2,
    callbacks = callbacks)

Epoch 1/45
125/125 [==============================] - 14s 66ms/step - loss: 217.1129 - mse: 72116.2109 - val_loss: 205.1624 - val_mse: 65634.7812
Epoch 2/45
125/125 [==============================] - 7s 55ms/step - loss: 165.1069 - mse: 42292.8594 - val_loss: 143.7738 - val_mse: 33988.8438
Epoch 3/45
125/125 [==============================] - 7s 57ms/step - loss: 152.5023 - mse: 36982.1094 - val_loss: 136.5674 - val_mse: 32045.8633
Epoch 4/45
125/125 [==============================] - 7s 58ms/step - loss: 144.2890 - mse: 33088.9492 - val_loss: 133.0314 - val_mse: 29136.1289
Epoch 5/45
125/125 [==============================] - 8s 62ms/step - loss: 132.8689 - mse: 28777.2578 - val_loss: 128.5932 - val_mse: 30298.6445
Epoch 6/45
125/125 [==============================] - 7s 59ms/step - loss: 132.3809 - mse: 28957.4844 - val_loss: 119.8063 - val_mse: 25232.2363
Epoch 7/45
125/125 [==============================] - 8s 60ms/step - loss: 135.0712 - mse: 29712.2949 - val_loss: 146.2145 - val_

125/125 [==============================] - 8s 67ms/step - loss: 117.9712 - mse: 23076.7930 - val_loss: 110.4914 - val_mse: 19019.7148
Epoch 28/45
125/125 [==============================] - 8s 66ms/step - loss: 117.7348 - mse: 23283.6953 - val_loss: 105.7524 - val_mse: 20763.5332
Epoch 29/45
125/125 [==============================] - 9s 74ms/step - loss: 122.9749 - mse: 24873.1270 - val_loss: 107.6068 - val_mse: 18417.9980
Epoch 30/45
125/125 [==============================] - 8s 63ms/step - loss: 117.1796 - mse: 22933.4746 - val_loss: 111.4741 - val_mse: 19058.5547
Epoch 31/45
125/125 [==============================] - 7s 59ms/step - loss: 117.0727 - mse: 22896.4766 - val_loss: 107.9415 - val_mse: 21026.1055
Epoch 32/45
125/125 [==============================] - 8s 66ms/step - loss: 119.0468 - mse: 23487.3320 - val_loss: 115.1147 - val_mse: 19977.7793
Epoch 33/45
125/125 [==============================] - 7s 59ms/step - loss: 118.4188 - mse: 23069.5449 - val_loss: 111.6848 - val_mse: 1

In [296]:
model.save_weights("h15")

In [83]:
Y_pred = model.predict([time_x_test, static_x_test])

32/32 [==============================] - 1s 5ms/step


In [84]:
se = 0
for i in range(len(Y_pred)):
    se+=(Y_pred[i]-Y_test[i])**2
se=se/1000
print(se)

[26809.83]


In [85]:
se**(1/2)

array([163.73708], dtype=float32)

In [151]:
from sklearn.metrics import mean_squared_error

In [455]:
Y_pr = [Y_pred[i][0] for i in range(len(Y_pred))]

In [456]:
mean_squared_error(Y_test, Y_pr)

25276.148

In [304]:

X_test = x_test

In [329]:
static_x_test = X_test.drop(time_columns, axis = 1)
time_x_test = X_test.drop(static_columns, axis = 1)


In [330]:
lstm_x_speed = []
lstm_x_dist = []

In [331]:
len(lstm_x_speed)

0

In [332]:
len(lstm_x_dist)

0

In [333]:
for i in df_test["Id"]:
    temp_df = nodes_df[nodes_df['Id'] == i]
    ls = temp_df["speed"].to_numpy()
    ls = np.pad(ls, (0, 300 - len(ls)%300), 'constant')
    ld = temp_df["distance"].to_numpy()
    ld = np.pad(ld, (0, 300 - len(ld)%300), 'constant')
    lstm_x_speed.append(ls)
    lstm_x_dist.append(ld)

In [334]:
lstm_x_speed = np.asarray(lstm_x_speed).astype('float32')
lstm_x_dist = np.asarray(lstm_x_dist).astype('float32')

In [335]:
static_x_test = static_x_test.drop(["Id"], axis = 1)
time_x_test = time_x_test.drop(["Id"], axis = 1)

In [336]:
Y_pred = model.predict([time_x_test, static_x_test, lstm_x_speed, lstm_x_dist])

32/32 [==============================] - 2s 21ms/step


In [337]:
df_test["Predicted"] = Y_pred

In [339]:
df_test = df_test[["Id", "Predicted"]]

In [340]:
df_test

,Id,Predicted
0,6198,668.584900
1,6417,635.973633
2,7054,655.983276
3,9628,615.632385
4,10283,650.400879
...,...,...
995,525706,535.278076
996,526604,549.520020
997,527213,521.269897
998,527520,350.894165


In [341]:
df_test.to_csv()

',Id,Predicted\r\n0,6198,668.5849\r\n1,6417,635.97363\r\n2,7054,655.9833\r\n3,9628,615.6324\r\n4,10283,650.4009\r\n5,10660,482.52188\r\n6,12778,527.26117\r\n7,15267,695.33453\r\n8,17648,490.5948\r\n9,20377,683.6954\r\n10,21710,533.40686\r\n11,23832,655.7658\r\n12,24693,704.44946\r\n13,28920,601.6003\r\n14,29889,657.6755\r\n15,30186,608.5209\r\n16,30596,570.9651\r\n17,30815,624.7533\r\n18,30958,481.61624\r\n19,31401,633.78253\r\n20,32452,694.843\r\n21,33885,565.35315\r\n22,34012,549.8318\r\n23,37794,678.1706\r\n24,38858,567.2301\r\n25,38961,384.13452\r\n26,39912,627.15857\r\n27,43686,628.8891\r\n28,43792,427.33493\r\n29,44405,599.65234\r\n30,44821,549.2839\r\n31,45083,612.3218\r\n32,45315,529.1186\r\n33,47093,663.951\r\n34,47190,573.8474\r\n35,47549,651.57697\r\n36,49333,600.3849\r\n37,50669,552.9178\r\n38,53294,588.0896\r\n39,54019,533.86786\r\n40,54558,615.95435\r\n41,55170,599.46735\r\n42,55846,503.6539\r\n43,56337,604.259\r\n44,57502,558.4912\r\n45,57569,663.6715\r\n46,57913,518.551